In [ ]:
import asyncio
from util import parse_url

'''
# freenode irc server example info
SERVER = 'chat.freenode.net:6666'
CHANNEL = '#hojong'
NICK = 'hjbot'
PASS = ''
'''

# twitch IRC server info
SERVER = 'irc.chat.twitch.tv'
CHANNEL = '#...' # set the channel you want to join
NICK = '...' # set your nickname
PASS = 'oauth:token' # you can get it from https://twitchapps.com/tmi/

class IRCClient(asyncio.Protocol):
    def __init__(self, CHANNEL, NICK, PASS, loop):
        self.channel = CHANNEL
        self.nick = NICK
        self.password = PASS
        self.loop = loop

    def connection_made(self, transport):
        msg_list = []
        if self.password and len(self.password) > 0:
            msg_list.append('PASS %s\r\n' % self.password)
        msg_list.append('NICK %s\r\n' % self.nick)
        msg_list.append('USER ... 0 * :...\r\n')
        msg_list.append('JOIN %s\r\n' % self.channel)
        
        for msg in msg_list:
            print('*** SENDED:', msg, end='')
            transport.write(msg.encode())

    def data_received(self, data):
        packet = data.decode().split(' ')
        nickname = (packet[0].split('!'))[0][1:]
        command = packet[1]
        
        if command == 'PRIVMSG':
            chat = packet[3][1:].replace('\r\n', '')
            print('%s: %s' % (nickname, chat))

    def connection_lost(self, exc):
        print('The server closed the connection')
        print('Stop the event loop')
        self.loop.stop()

def main():
    host, port = parse_url(SERVER)
    assert (PASS != 'oauth:token'), 'you must set PASS'
    print('*** SETTING INFO: %s %d %s %s %s' % (host, port, CHANNEL, NICK, PASS))
    
    loop = asyncio.get_event_loop()
    coroutine = loop.create_connection(lambda: IRCClient(CHANNEL, NICK, PASS, loop), host, port)

    try:
        loop.run_until_complete(coroutine)
        loop.run_forever()
        loop.close()
    except Exception as e:
        print(e)
    
if __name__ == '__main__':
    main()